# Programming Assignment: Social Network Database

## Welcome!

In this module you've been learning how LLMs can help with many common tasks when working with databases. In this assignment you'll have an opportunity to put those skills into practice. You will be working with a database designed to represent a simple social network. Here's the tasks you'll need to complete:

1. Write code that reads data from a provided CSV file into the database
2. Develop functions that query the database

### Submission and Grading

This notebook will be graded based on four functions that you write. You can find starter code for all four functions below. Unit tests are provided to help you test your work.

### Working with the LLM:

- **GPT-4o is available:** You can use whichever LLM you prefer on this assignment, but [GPT-4o has been made avaiable](https://www.coursera.org/learn/ai-powered-software-and-system-design/ungradedLab/rSPHu/gpt-4o-environment-for-assignment-to-use-alongside-the-programming-assignment) in the ungraded lab that follows this assignment
- **Provide Context to Your LLM:** The LLM will need the context of the problem you're working on and the code already available to you in order to help develop solutions.
- **Work Iteratively and Test as You Go:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful, test the code it provides, and iterate as you work towards a solution.

### Necessary imports

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os
import pandas as pd
import json

In [2]:
import unittests

### Database Schema

The function below defines the schema for the database you will be using in this assignment. The database represents a social network made up of people who can be friends with one another and join clubs. You cannot edit this function but should understand how it works and the database schema it defines. **Read this function carefully and share it with your LLM** to make sure you understand the schema before moving on.

In [3]:
# Creates the database for the social network

def create_database():
    Base = declarative_base()

    friendships = Table('friendships', Base.metadata,
                        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True))

    club_members = Table('club_members', Base.metadata,
                         Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                         Column('club_id', Integer, ForeignKey('clubs.id'), primary_key=True))

    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=id == friendships.c.person_id,
                               secondaryjoin=id == friendships.c.friend_id)
        clubs = relationship("Club", secondary=club_members, back_populates="members")

    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
        members = relationship("Person", secondary=club_members, back_populates="clubs")

    if os.path.exists("social_network.db"):
        os.remove("social_network.db")
    engine = create_engine(f'sqlite:///{"social_network.db"}', echo=False)
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    return session, Club, Person, friendships, club_members

### Exercise 1: Load the Database from a CSV

The database defined in the `create_database` function does not yet contain any data. You'll need to write the code for the `load_data_from_csv` function below to populate the database. To do that, you'll use the `members.csv` file which is available in this lab in the same directory as this notebook. The data is stored in a single table with one row for each person. The Friendships column contains the IDs of everyone the person in that row considers a friend. The Clubs column contains the names of each club the person in that row is a part of.

A few important notes about this exercise:

* In order for your function to be graded properly, do not change the function name or its parameters.
* You shouldn't need to import additional libraries, and this can sometimes cause errors with the grader. If you absolutely must, do so within the same cell as the `load_data_from_csv` function.

In [4]:
# The members.csv file contains 20 people. The code below will show you the first 5 rows to help you understand the format of the CSV file

pd.read_csv("members.csv", converters = {'Friendships': eval, "Clubs": eval}).head()

,ID,Name,Surname,Location,Age,Gender,Friendships,Clubs
0,0,John,Rocha,"948 Christian Park Apt. 683 New Christopher, M...",57,Male,"[4, 10, 12, 5, 19, 14]",[Fitness Club]
1,1,William,Ruiz,"09477 Paul Station Williamsmouth, NM 93899",42,Female,"[9, 15, 17, 10, 3]","[Travel Club, Art Club, Cooking Club]"
2,2,Jackie,Mccullough,"903 Keller River Suite 149 Davidside, IN 11476",36,Non-binary,"[6, 4, 10, 9]","[Hiking Club, Art Club]"
3,3,Michael,Powell,"7528 Brenda Mills Jenniferstad, RI 38209",67,Female,"[11, 7, 13, 6, 14, 2]","[Hiking Club, Chess Club, Gaming Club, Book Cl..."
4,4,Scott,Boyd,"545 Evelyn Shores Apt. 744 North Craigchester,...",47,Female,"[15, 17, 1, 11, 0, 10, 5]",[Gaming Club]


In [11]:
def load_data_from_csv(session, Club, Person, friendships, club_members, csv_path="members.csv"):
    # Step 1: Clear existing data from all relevant tables
    session.query(Person).delete()
    session.query(Club).delete()
    session.query(friendships).delete()
    session.query(club_members).delete()
    session.commit()  # Commit the deletion of all existing records

    # Load the CSV data
    df = pd.read_csv(csv_path, converters={'Friendships': eval, "Clubs": eval})

    # Mapping of club names to Club objects
    club_map = {}

    # Step 2: Insert people into the database
    for _, row in df.iterrows():
        # Combine Name and Surname to create the full name
        full_name = f"{row['Name']} {row['Surname']}"
        person = Person(id=row['ID'], name=full_name, age=row['Age'], gender=row['Gender'], location=row['Location'])
        session.add(person)
        session.commit()

        # Step 3: Insert clubs into the database and establish memberships
        for club_name in row['Clubs']:
            if club_name not in club_map:
                club = Club(description=club_name)
                session.add(club)
                session.commit()
                club_map[club_name] = club
            else:
                club = club_map[club_name]
            person.clubs.append(club)

        session.commit()

    # Step 4: Establish friendships
    for _, row in df.iterrows():
        person = session.query(Person).get(row['ID'])
        for friend_id in row['Friendships']:
            friend = session.query(Person).get(friend_id)
            if friend:
                person.friends.append(friend)

        session.commit()

In [12]:
# The code below creates the database and reads in the data 
session, Club, Person, friendships, club_members = create_database()
load_data_from_csv(session, Club, Person, friendships, club_members, "members.csv")

# If your load_data_from_csv function is working correctly, then you should have read in data correctly into all four tables in the database.

print_amount = 3

# Print first 3 persons
print("=== All Persons ===")
people = session.query(Person).all()
for person in people[:print_amount ]:
    print(f"ID: {person.id}, Name: {person.name}, Age: {person.age}, Gender: {person.gender}, Location: {person.location}")

# Print first 3 clubs and their members
print("\n=== All Clubs and their Members ===")
clubs = session.query(Club).all()
for club in clubs[:print_amount ]:
    print(f"Club ID: {club.id}, Description: {club.description}, Members: {[member.name for member in club.members]}")

# Print friendships of first three persons
print("\n=== Friendships ===")
for person in people[:print_amount ]:
    friends = [friend.name for friend in person.friends]
    print(f"{person.name}'s Friends: {friends}")

/tmp/ipykernel_111/2147410242.py:38: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  person = session.query(Person).get(row['ID'])
/tmp/ipykernel_111/2147410242.py:40: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  friend = session.query(Person).get(friend_id)


=== All Persons ===
ID: 0, Name: John Rocha, Age: 57, Gender: Male, Location: 948 Christian Park Apt. 683 New Christopher, MN 06675
ID: 1, Name: William Ruiz, Age: 42, Gender: Female, Location: 09477 Paul Station Williamsmouth, NM 93899
ID: 2, Name: Jackie Mccullough, Age: 36, Gender: Non-binary, Location: 903 Keller River Suite 149 Davidside, IN 11476

=== All Clubs and their Members ===
Club ID: 1, Description: Fitness Club, Members: ['John Rocha', 'Amanda Norris', 'Michael Clark', 'Christina Murphy']
Club ID: 2, Description: Travel Club, Members: ['William Ruiz', 'Michael Powell', 'Becky Peterson', 'Nicholas Harrington', 'Luis Kim', 'Nathan Mendez']
Club ID: 3, Description: Art Club, Members: ['William Ruiz', 'Jackie Mccullough', 'Amanda Norris', 'Becky Peterson', 'Mark Allen', 'Brian Mays', 'Eric Dougherty', 'Nathan Mendez']

=== Friendships ===
John Rocha's Friends: ['Scott Boyd', 'Andrew Williams', 'Nicholas Harrington', 'Christina Murphy', 'Luis Kim', 'Mark Mcintyre']
William Ru

Expected output:
```
=== All Persons ===
ID: 0, Name: John Rocha, Age: 57, Gender: Male, Location: 948 Christian Park Apt. 683 New Christopher, MN 06675
ID: 1, Name: William Ruiz, Age: 42, Gender: Female, Location: 09477 Paul Station Williamsmouth, NM 93899
ID: 2, Name: Jackie Mccullough, Age: 36, Gender: Non-binary, Location: 903 Keller River Suite 149 Davidside, IN 11476

=== All Clubs and their Members ===
Club ID: 1, Description: Fitness Club, Members: ['John Rocha', 'Amanda Norris', 'Michael Clark', 'Christina Murphy']
Club ID: 2, Description: Travel Club, Members: ['William Ruiz', 'Michael Powell', 'Becky Peterson', 'Nicholas Harrington', 'Luis Kim', 'Nathan Mendez']
Club ID: 3, Description: Art Club, Members: ['William Ruiz', 'Jackie Mccullough', 'Amanda Norris', 'Becky Peterson', 'Mark Allen', 'Brian Mays', 'Eric Dougherty', 'Nathan Mendez']

=== Friendships ===
John Rocha's Friends: ['Scott Boyd', 'Andrew Williams', 'Nicholas Harrington', 'Christina Murphy', 'Luis Kim', 'Mark Mcintyre']
William Ruiz's Friends: ['Michael Powell', 'Mark Allen', 'Nicholas Harrington', 'Brian Mays', 'Nathan Mendez']
Jackie Mccullough's Friends: ['Scott Boyd', 'Amanda Norris', 'Mark Allen', 'Nicholas Harrington']
```

In [13]:
unittests.test_load_data_from_csv(load_data_from_csv)

/tmp/ipykernel_111/2147410242.py:38: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  person = session.query(Person).get(row['ID'])
/tmp/ipykernel_111/2147410242.py:40: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  friend = session.query(Person).get(friend_id)


 All tests passed!


### Exercise 2: Get a Club's Members

Assuming your data has been loaded correctly, you should now be able to query the data in your database. For this exercise, write a function called `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [14]:
def get_club_members(session, club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - session: The SQLAlchemy session for database queries.
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the Club table to find the club with the given description
    club = session.query(Club).filter_by(description=club_description).first()
    
    if club:
        # Return the list of members associated with this club
        return club.members
    else:
        # If no club is found with the given description, return an empty list
        return []

# Example usage:
# session, Club, Person, friendships, club_members = create_database()
# load_data_from_csv(session, Club, Person, friendships, club_members)
# members = get_club_members(session, "Hiking Club")
# for member in members:
#     print(member.name)

In [15]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session, "Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Members of the Hiking Club:
- Jackie Mccullough, Age: 36, Location: 903 Keller River Suite 149 Davidside, IN 11476
- Michael Powell, Age: 67, Location: 7528 Brenda Mills Jenniferstad, RI 38209
- Amanda Norris, Age: 27, Location: 8446 Thomas Crossing Suite 184 West Jacobshire, CA 41663
- Michael Clark, Age: 39, Location: 688 Sean River Apt. 585 Amandatown, AZ 76721
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034


Expected output:
```
Members of the Hiking Club:
- Jackie Mccullough, Age: 36, Location: 903 Keller River Suite 149 Davidside, IN 11476
- Michael Powell, Age: 67, Location: 7528 Brenda Mills Jenniferstad, RI 38209
- Amanda Norris, Age: 27, Location: 8446 Thomas Crossing Suite 184 West Jacobshire, CA 41663
- Michael Clark, Age: 39, Location: 688 Sean River Apt. 585 Amandatown, AZ 76721
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034

```

In [16]:
unittests.test_get_club_members(load_data_from_csv, get_club_members)

/tmp/ipykernel_111/2147410242.py:38: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  person = session.query(Person).get(row['ID'])
/tmp/ipykernel_111/2147410242.py:40: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  friend = session.query(Person).get(friend_id)


 All tests passed!


### Exercise 3: Get a Person's Friends

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all the people they consider to be friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [17]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - session: The SQLAlchemy session object used to query the database.
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter_by(name=person_name).first()
    
    if person:
        # Return the list of friends associated with this person
        return person.friends
    else:
        # If no person is found with the given name, return an empty list
        return []

# Example usage:
# session, Club, Person, friendships, club_members = create_database()
# load_data_from_csv(session, Club, Person, friendships, club_members)
# friends = get_friends_of_person(session, "John Rocha")
# for friend in friends:
#     print(friend.name)

In [18]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "John Rocha"

john_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of John Rocha
print(f"Friends of {name}:")
for friend in john_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Friends of John Rocha:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Andrew Williams, Age: 24, Location: USCGC Jacobs FPO AE 78539
- Nicholas Harrington, Age: 52, Location: 97681 Hernandez Villages Suite 344 Elizabethborough, UT 44717
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Mark Mcintyre, Age: 22, Location: 5635 Jennifer Freeway Suite 436 Newmanberg, IL 05315


Expected output:
```
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Andrew Williams, Age: 24, Location: USCGC Jacobs FPO AE 78539
- Nicholas Harrington, Age: 52, Location: 97681 Hernandez Villages Suite 344 Elizabethborough, UT 44717
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Mark Mcintyre, Age: 22, Location: 5635 Jennifer Freeway Suite 436 Newmanberg, IL 05315
```

In [19]:
unittests.test_get_friends_of_person(load_data_from_csv, get_friends_of_person)

/tmp/ipykernel_111/2147410242.py:38: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  person = session.query(Person).get(row['ID'])
/tmp/ipykernel_111/2147410242.py:40: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  friend = session.query(Person).get(friend_id)


 All tests passed!


### Exercise 4: Get Everyone that Considers a Person a Friend

Now write a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [20]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - session: The SQLAlchemy session object used to query the database.
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter_by(name=person_name).first()
    
    if not person:
        # If no person is found with the given name, return an empty list
        return []

    # Find all people who consider this person as a friend
    persons_who_consider_friend = session.query(Person).filter(Person.friends.contains(person)).all()

    return persons_who_consider_friend

# Example usage:
# session, Club, Person, friendships, club_members = create_database()
# load_data_from_csv(session, Club, Person, friendships, club_members)
# admirers = get_persons_who_consider_them_friend(session, "John Rocha")
# for admirer in admirers:
#     print(admirer.name)

In [21]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'John Rocha'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

People who consider John Rocha as their friend:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Brian Mays, Age: 61, Location: 80208 Parker Glen Harrisland, PW 54882
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034


Expected output:
```
People who consider John Rocha as their friend:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Brian Mays, Age: 61, Location: 80208 Parker Glen Harrisland, PW 54882
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034
```

In [22]:
unittests.test_get_persons_who_consider_them_friend(load_data_from_csv, get_persons_who_consider_them_friend)

/tmp/ipykernel_111/2147410242.py:38: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  person = session.query(Person).get(row['ID'])
/tmp/ipykernel_111/2147410242.py:40: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  friend = session.query(Person).get(friend_id)


 All tests passed!


Once you've written all four methods you're ready to submit. **Make sure you save your work before submitting** so that the grader will assess the most recent version of your assignment.

Congratulations on finishing this assignment!